# Using Kinematics from Reachy's URDF file

## Creating the Chain from the URDF

We use the [Ikpy](https://github.com/Phylliade/ikpy) library to create the kinematic chain from the URDF. You have to specify:

* the path to the URDF file
* the base element of your chain
* the translation vector for the end tip

In [1]:
from ikpy.chain import Chain

urdf_file = '../dev/reachy/software/reachy/reachy.URDF'

reachy_kinematic_chain = Chain.from_urdf_file(urdf_file, base_elements=['base'], last_link_vector=[0, 0, -0.02409])

You can check the created chain:

In [2]:
for link in reachy_kinematic_chain.links:
    print(link)

Link name=Base link
URDF Link shoulder_pitch :
    Translation : [ 0.     -0.0887  0.    ]
    Orientation : [ 0.  0.  0.]
    Rotation : [ 0.  1.  0.]
URDF Link shoulder_roll :
    Translation : [ 0.  0.  0.]
    Orientation : [ 0.  0.  0.]
    Rotation : [ 1.  0.  0.]
URDF Link arm_yaw :
    Translation : [ 0.  0.  0.]
    Orientation : [ 0.  0.  0.]
    Rotation : [ 0.  0.  1.]
URDF Link elbow_pitch :
    Translation : [ 0.       0.      -0.30745]
    Orientation : [ 0.  0.  0.]
    Rotation : [ 0.  1.  0.]
URDF Link forearm_yaw :
    Translation : [ 0.  0.  0.]
    Orientation : [ 0.  0.  0.]
    Rotation : [ 0.  0.  1.]
URDF Link wrist_pitch :
    Translation : [ 0.       0.      -0.22415]
    Orientation : [ 0.  0.  0.]
    Rotation : [ 0.  1.  0.]
URDF Link wrist_roll :
    Translation : [ 0.       0.      -0.03243]
    Orientation : [ 0.  0.  0.]
    Rotation : [ 1.  0.  0.]
URDF Link last_joint :
    Translation : [ 0.       0.      -0.02409]
    Orientation : [0 0 0]
    Rota

## Forward kinematic

You can then call the forward kinematic method by specifying all joint angle (in radians). You have to specify 9 joints in the case of Reachy: the 7 defined joints plus one virtual before and one virtual at the end.

For instance, when setting all joints to 0 except the elbow pitch at $\pi /2$:

In [3]:
import numpy

target = reachy_kinematic_chain.forward_kinematics(joints=[0, 0, 0, 0, -numpy.pi / 2, 0, 0, 0, 0])
print(target)

[[  6.12323400e-17   0.00000000e+00  -1.00000000e+00   2.80670000e-01]
 [  0.00000000e+00   1.00000000e+00   0.00000000e+00  -8.87000000e-02]
 [  1.00000000e+00   0.00000000e+00   6.12323400e-17  -3.07450000e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


You can then extract the rotation matrix:

In [4]:
rot = target[:3,:3]
print(rot)

[[  6.12323400e-17   0.00000000e+00  -1.00000000e+00]
 [  0.00000000e+00   1.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   6.12323400e-17]]


And the translation vector:

In [5]:
trans = target[:3, -1]
print(trans)

[ 0.28067 -0.0887  -0.30745]


## Inverse kinematics

You can also use the inverse kinematics to find the joint angles from a target. The target must be passed as a 4x4 homogeneous matrix.

In [6]:
rot = numpy.eye(3)
trans = [0.28, -0.09, -0.30]

target = numpy.eye(4)
target[:3, :3] = rot
target[:3, 3] = trans

In [7]:
joints = reachy_kinematic_chain.inverse_kinematics(target)
print(joints)

[  0.00000000e+00  -7.14762238e-03  -2.55830602e-03  -1.86245948e-03
  -1.52041605e+00  -1.98370222e-04  -3.51105804e-01  -2.51860806e-04
   0.00000000e+00]


You can also specify an initial position for the inverse kinematics optimisation. This will drastically speed up the process and help find better solutions.

In [8]:
joints = reachy_kinematic_chain.inverse_kinematics(target, initial_position=[0, 0, 0, 0, -numpy.pi/2, 0, 0, 0, 0])
joints

array([  0.00000000e+00,   1.85346225e-03,  -2.32089073e-03,
        -2.14040571e-03,  -1.59807829e+00,   1.12758811e-06,
        -5.50495729e-03,  -1.84077261e-04,   0.00000000e+00])

## Use it with reachy

In [9]:
from reachy import Reachy

reachy = Reachy()

In [10]:
def goto(joints):
    for m, j in zip(reachy.motors, joints[1:-1]):
        m.goal_position = numpy.rad2deg(j)

In [13]:
for m in reachy.motors:
    m.compliant = False
    m.moving_speed = 50

In [14]:
goto(joints)

### You can find more information on Ikpy documentation: https://github.com/Phylliade/ikpy